# EMA(20,50,200) + ADX Swing Backtest — Trailing Stop & 5% Risk

Swing-oriented backtest (daily bars) with the following user-requested changes:

- **Swing mode**: entries/exits are executed at next-day open (no intraday entry/exit fills). This avoids 'intraday' exits on the same candle.
- **Max loss per trade**: use 5% of capital-at-risk per trade by default (changeable in the parameters cell).
- **Max holding period**: 10 days maximum holding before forced exit.
- **ATR-based trailing stop**: a trailing stop initialized at entry (entry - ATR*mult) which moves favorably each day (longs only move upward). If the day's low/high breaches the trailing level, we exit at the next day's open.

Edit parameters in the Parameters cell and run top-to-bottom.


In [26]:
# ----------------------------
# PARAMETERS (edit these)
# ----------------------------
STOCKS = ['SBICARD.NS', 'BDL.NS', 'INDHOTEL.NS', 'BSE.NS', 'NYKAA.NS', 'BAJFINANCE.NS', 'PAYTM.NS', 'SOLARINDS.NS', 'CHOLAFIN.NS', 'UNITDSPR.NS', 'DIVISLAB.NS', 'MUTHOOTFIN.NS', 'BHARTIARTL.NS', 'ICICIBANK.NS', 'MAZDOCK.NS', 'SHREECEM.NS', 'DIXON.NS', 'PERSISTENT.NS', 'SRF.NS', 'TVSMOTOR.NS', 'SBILIFE.NS', 'MAXHEALTH.NS', 'MFSL.NS', 'COFORGE.NS', 'HDFCLIFE.NS', 'INDIGO.NS', 'KOTAKBANK.NS', 'HDFCBANK.NS', 'BEL.NS', 'BAJAJFINSV.NS']
START_DATE = '2015-01-01'
END_DATE = '2025-01-01'                # change as needed
TIMEFRAME = '1d'                       # daily bars for swing trading
ADX_THRESHOLD = 25                     # ADX filter
ATR_LENGTH = 14
ATR_MULTIPLIER = 1.0                   # used to set initial stop = entry +/- ATR*mult
TRAILING_ATR_MULT = 1.0                # trailing uses ATR * this multiplier
RISK_PER_TRADE = 0.05                  # max loss per trade = 5% of capital (user requested)
INITIAL_CAPITAL = 100000.0             # starting capital in INR (or chosen currency)
MAX_HOLD_DAYS = 10                     # user requested max 10 days holding (swing)
TAKE_PROFIT_RR = None                  # optional fixed TP (None by default for swing)
USE_SHORTS = True                      # enable short trades
BROKERAGE_PER_TRADE = 20.0             # flat brokerage per trade (INR) – adjust for your broker
SLIPPAGE_PCT = 0.0005                  # 0.05% slippage per trade
PRINT_PROGRESS = True                  # show progress while backtesting
# ----------------------------


In [27]:
# Install required libraries (run this cell once)
# !pip install yfinance matplotlib pandas numpy --quiet

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
import math


In [28]:
# Robust data download helper (normalizes column names)
def download_stock(ticker, start, end, interval='1d', threads=True):
    df = yf.download(ticker, start=start, end=end, interval=interval, progress=False, multi_level_index=False, auto_adjust=True)
    if df is None or df.empty:
        raise ValueError(f'No data for {ticker} from {start} to {end}')
    # Normalize columns
    col_map = {}
    cols = list(df.columns)
    for c in cols:
        lc = c.lower()
        if 'open' in lc:
            col_map[c] = 'Open'
        elif 'high' in lc:
            col_map[c] = 'High'
        elif 'low' in lc:
            col_map[c] = 'Low'
        elif 'close' in lc and 'adj' not in lc:
            col_map[c] = 'Close'
        elif 'adj' in lc and 'close' in lc:
            col_map[c] = 'Adj_Close'
        elif 'volume' in lc:
            col_map[c] = 'Volume'
    df = df.rename(columns=col_map)
    if 'Adj_Close' not in df.columns:
        if 'Close' in df.columns:
            df['Adj_Close'] = df['Close']
        else:
            raise KeyError("Downloaded data missing Close/Adj Close columns.")
    required = ['Open','High','Low','Close','Adj_Close','Volume']
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise KeyError(f"Missing columns after normalization: {missing}. Found: {df.columns.tolist()}")
    df = df[['Open','High','Low','Close','Adj_Close','Volume']].copy()
    if not isinstance(df.index, pd.DatetimeIndex):
        df.index = pd.to_datetime(df.index)
    return df

# quick check (safe)
print('Sanity check: downloading first ticker...')
_sample = download_stock(STOCKS[0], START_DATE, END_DATE, TIMEFRAME)
print('Downloaded rows:', len(_sample))
_sample.head()


Sanity check: downloading first ticker...
Downloaded rows: 1187


,Open,High,Low,Close,Adj_Close,Volume
Date,,,,,,
2020-03-16,651.218506,743.827492,646.292496,671.316650,671.316650,60887005
2020-03-17,680.774410,736.881662,678.804007,720.083984,720.083984,15145135
2020-03-18,740.970256,757.620191,669.937230,679.001099,679.001099,8949832
2020-03-19,650.233251,706.389759,640.381232,681.808960,681.808960,6432046
2020-03-20,695.798814,719.197358,680.035584,713.778748,713.778748,6492226


In [29]:
# Custom indicators (EMA, ATR, ADX) implemented with pandas/numpy (Wilder smoothing for ATR/ADX)
def ema(series, length):
    return series.ewm(span=length, adjust=False).mean()

def true_range(df):
    high = df['High']
    low = df['Low']
    prev_close = df['Close'].shift(1)
    tr1 = high - low
    tr2 = (high - prev_close).abs()
    tr3 = (low - prev_close).abs()
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    return tr

def atr(df, length=14):
    tr = true_range(df)
    return tr.ewm(alpha=1/length, adjust=False).mean()

def directional_movements(df):
    up_move = df['High'] - df['High'].shift(1)
    down_move = df['Low'].shift(1) - df['Low']
    plus_dm = np.where((up_move > down_move) & (up_move > 0), up_move, 0.0)
    minus_dm = np.where((down_move > up_move) & (down_move > 0), down_move, 0.0)
    plus_dm = pd.Series(plus_dm, index=df.index)
    minus_dm = pd.Series(minus_dm, index=df.index)
    return plus_dm, minus_dm

def adx(df, length=14):
    tr = true_range(df)
    atr_series = tr.ewm(alpha=1/length, adjust=False).mean()
    plus_dm, minus_dm = directional_movements(df)
    plus_dm_smooth = plus_dm.ewm(alpha=1/length, adjust=False).mean()
    minus_dm_smooth = minus_dm.ewm(alpha=1/length, adjust=False).mean()
    di_plus = 100 * (plus_dm_smooth / atr_series).replace([np.inf, -np.inf], 0).fillna(0)
    di_minus = 100 * (minus_dm_smooth / atr_series).replace([np.inf, -np.inf], 0).fillna(0)
    dx = ( (di_plus - di_minus).abs() / (di_plus + di_minus).replace(0, np.nan) ) * 100
    dx = dx.replace([np.inf, -np.inf], np.nan).fillna(0)
    adx_series = dx.ewm(alpha=1/length, adjust=False).mean()
    out = pd.DataFrame({'ADX': adx_series, 'DI+': di_plus, 'DI-': di_minus}, index=df.index)
    return out

def add_indicators_custom(df):
    df = df.copy()
    df['EMA20'] = ema(df['Close'], 20)
    df['EMA50'] = ema(df['Close'], 50)
    df['EMA200'] = ema(df['Close'], 200)
    df['ATR'] = atr(df, ATR_LENGTH)
    adx_df = adx(df, 14)
    df['ADX'] = adx_df['ADX']
    df['DI+'] = adx_df['DI+']
    df['DI-'] = adx_df['DI-']
    return df

# quick test
_sample = add_indicators_custom(_sample)
_sample[['Close','EMA20','EMA50','EMA200','ATR','ADX']].tail()


,Close,EMA20,EMA50,EMA200,ATR,ADX
Date,,,,,,
2024-12-24,693.827026,705.238429,709.574453,726.404021,14.892501,22.809076
2024-12-26,677.176758,702.565889,708.303956,725.914198,15.313603,23.710837
2024-12-27,673.288391,699.777556,706.930796,725.390558,14.992464,24.742958
2024-12-30,667.505676,696.704043,705.384713,724.814589,14.761919,26.082214
2024-12-31,661.872437,693.386748,703.678349,724.188299,14.355563,27.530573


In [30]:
# Safe signal generation for swing strategy
def generate_signals_safe(df):
    df = df.copy()
    df['EMA20_gt_EMA50'] = (df['EMA20'] > df['EMA50']).fillna(False)
    df['EMA20_gt_EMA50_prev'] = df['EMA20_gt_EMA50'].shift(1).fillna(False)
    df['long_cross'] = (df['EMA20_gt_EMA50'] == True) & (df['EMA20_gt_EMA50_prev'] == False)
    df['short_cross'] = (df['EMA20_gt_EMA50'] == False) & (df['EMA20_gt_EMA50_prev'] == True)
    df['price_above_200'] = (df['Close'] > df['EMA200']).fillna(False)
    df['price_below_200'] = (df['Close'] < df['EMA200']).fillna(False)
    df['adx_ok'] = (df['ADX'] > ADX_THRESHOLD).fillna(False)
    df['signal_long'] = df['long_cross'] & df['price_above_200'] & df['adx_ok']
    df['signal_short'] = df['short_cross'] & df['price_below_200'] & df['adx_ok']
    return df

# test
_sample2 = generate_signals_safe(_sample)
_sample2[['Close','EMA20','EMA50','EMA200','ADX','signal_long','signal_short']].tail(8)


/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_94721/3118873791.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['EMA20_gt_EMA50_prev'] = df['EMA20_gt_EMA50'].shift(1).fillna(False)


,Close,EMA20,EMA50,EMA200,ADX,signal_long,signal_short
Date,,,,,,,
2024-12-19,701.304688,710.702302,712.139504,727.531884,21.605207,False,False
2024-12-20,684.953552,708.250040,711.073388,727.108219,21.909683,False,False
2024-12-23,689.240723,706.439629,710.217206,726.731428,22.192410,False,False
2024-12-24,693.827026,705.238429,709.574453,726.404021,22.809076,False,False
2024-12-26,677.176758,702.565889,708.303956,725.914198,23.710837,False,False
2024-12-27,673.288391,699.777556,706.930796,725.390558,24.742958,False,False
2024-12-30,667.505676,696.704043,705.384713,724.814589,26.082214,False,False
2024-12-31,661.872437,693.386748,703.678349,724.188299,27.530573,False,False


In [31]:
# Backtester (swing behavior, trailing stop, next-day open fills)
from collections import namedtuple
Trade = namedtuple('Trade', ['ticker','entry_date','entry_price','side','size','initial_stop','trailing_stop','exit_date','exit_price','pnl','reason'])

def run_backtest_for_symbol_swing(df, initial_capital=INITIAL_CAPITAL):
    equity = initial_capital
    cash = initial_capital
    position = None
    trades = []
    equity_curve = []
    peak_equity = initial_capital
    drawdowns = []

    for i in range(1, len(df)-1):
        row = df.iloc[i]
        next_row = df.iloc[i+1]

        # Entry logic (enter at next open after signal)
        if position is None:
            if row.get('signal_long', False):
                entry_price = next_row['Open'] * (1 + SLIPPAGE_PCT)
                atr = row['ATR'] if not np.isnan(row['ATR']) else 0.0
                initial_stop = entry_price - ATR_MULTIPLIER * atr
                # Cap risk: ensure risk amount per trade <= RISK_PER_TRADE * equity
                risk_amount = equity * RISK_PER_TRADE
                risk_per_share = entry_price - initial_stop
                if risk_per_share <= 0: 
                    continue
                size_shares = math.floor(risk_amount / risk_per_share)
                if size_shares <= 0:
                    continue
                # initialize trailing stop same as initial stop
                trailing_stop = entry_price - TRAILING_ATR_MULT * atr
                position = {'side':'long','entry_price':entry_price,'size':size_shares,'initial_stop':initial_stop,
                            'trailing_stop':trailing_stop,'entry_date':next_row.name,'held_days':0}
                cash -= entry_price * size_shares + BROKERAGE_PER_TRADE
            elif USE_SHORTS and row.get('signal_short', False):
                entry_price = next_row['Open'] * (1 - SLIPPAGE_PCT)
                atr = row['ATR'] if not np.isnan(row['ATR']) else 0.0
                initial_stop = entry_price + ATR_MULTIPLIER * atr
                risk_amount = equity * RISK_PER_TRADE
                risk_per_share = initial_stop - entry_price
                if risk_per_share <= 0:
                    continue
                size_shares = math.floor(risk_amount / risk_per_share)
                if size_shares <= 0:
                    continue
                trailing_stop = entry_price + TRAILING_ATR_MULT * atr
                position = {'side':'short','entry_price':entry_price,'size':size_shares,'initial_stop':initial_stop,
                            'trailing_stop':trailing_stop,'entry_date':next_row.name,'held_days':0}
                cash -= BROKERAGE_PER_TRADE
        else:
            # Manage position: update trailing stop and check exit conditions.
            exit_flag = False
            exit_price = None
            reason = None

            # update held days
            position['held_days'] += 1

            if position['side'] == 'long':
                # update trailing_stop to max(previous, close - ATR*TRAILING_ATR_MULT)
                atr = row['ATR'] if not np.isnan(row['ATR']) else 0.0
                new_trailing = max(position['trailing_stop'], row['Close'] - TRAILING_ATR_MULT * atr)
                position['trailing_stop'] = new_trailing
                # if today's low breaches trailing stop -> exit at next open
                if row['Low'] <= position['trailing_stop']:
                    exit_flag = True
                    exit_price = next_row['Open'] * (1 - SLIPPAGE_PCT)
                    reason = 'trailing_stop_hit'
                # forced time exit
                elif MAX_HOLD_DAYS and position['held_days'] >= MAX_HOLD_DAYS:
                    exit_flag = True
                    exit_price = next_row['Open'] * (1 - SLIPPAGE_PCT)
                    reason = 'time_exit'
                # ema-exit (optional): EMA20 crossing below EMA50 -> exit next open
                elif row['EMA20'] < row['EMA50']:
                    exit_flag = True
                    exit_price = next_row['Open'] * (1 - SLIPPAGE_PCT)
                    reason = 'ema_exit'
            else: # short
                atr = row['ATR'] if not np.isnan(row['ATR']) else 0.0
                new_trailing = min(position['trailing_stop'], row['Close'] + TRAILING_ATR_MULT * atr)
                position['trailing_stop'] = new_trailing
                if row['High'] >= position['trailing_stop']:
                    exit_flag = True
                    exit_price = next_row['Open'] * (1 + SLIPPAGE_PCT)
                    reason = 'trailing_stop_hit'
                elif MAX_HOLD_DAYS and position['held_days'] >= MAX_HOLD_DAYS:
                    exit_flag = True
                    exit_price = next_row['Open'] * (1 + SLIPPAGE_PCT)
                    reason = 'time_exit'
                elif row['EMA20'] > row['EMA50']:
                    exit_flag = True
                    exit_price = next_row['Open'] * (1 + SLIPPAGE_PCT)
                    reason = 'ema_exit'

            if exit_flag:
                if position['side'] == 'long':
                    exit_val = exit_price * position['size']
                    entry_val = position['entry_price'] * position['size']
                    pnl = exit_val - entry_val - BROKERAGE_PER_TRADE
                    cash += exit_val - BROKERAGE_PER_TRADE
                else:
                    entry_val = position['entry_price'] * position['size']
                    exit_val = exit_price * position['size']
                    pnl = entry_val - exit_val - BROKERAGE_PER_TRADE
                    cash += (entry_val - exit_val) - BROKERAGE_PER_TRADE

                equity = cash
                trades.append(Trade(ticker=None, entry_date=position['entry_date'], entry_price=position['entry_price'],
                                    side=position['side'], size=position['size'], initial_stop=position['initial_stop'],
                                    trailing_stop=position['trailing_stop'], exit_date=next_row.name, exit_price=exit_price, pnl=pnl, reason=reason))
                position = None

        # mark-to-market for equity curve using day's close price
        if position is None:
            mtm = cash
        else:
            if position['side'] == 'long':
                mtm = cash + row['Close'] * position['size']
            else:
                mtm = cash + (position['entry_price'] - row['Close']) * position['size']
        equity_curve.append({'date':row.name, 'equity':mtm})
        if mtm > peak_equity:
            peak_equity = mtm
        drawdowns.append((peak_equity - mtm) / peak_equity if peak_equity>0 else 0)

    eq_df = pd.DataFrame(equity_curve).set_index('date')
    trades_df = pd.DataFrame(trades)
    return trades_df, eq_df, (max(drawdowns)*100 if drawdowns else 0)

def backtest_universe_swing(tickers):
    all_trades = []
    equity_tracks = {}
    max_dd_per_symbol = {}
    for t in tickers:
        if PRINT_PROGRESS:
            print('Running:', t)
        df = download_stock(t, START_DATE, END_DATE, TIMEFRAME)
        df = add_indicators_custom(df)
        df = df.dropna(subset=['EMA20','EMA50','EMA200','ADX','ATR']).copy()
        if df.empty:
            if PRINT_PROGRESS:
                print(f"Not enough data/indicators for {t}; skipping.")
            continue
        df = generate_signals_safe(df)
        trades_df, eq_df, max_dd = run_backtest_for_symbol_swing(df, initial_capital=INITIAL_CAPITAL)
        trades_df['ticker'] = t
        all_trades.append(trades_df)
        equity_tracks[t] = eq_df
        max_dd_per_symbol[t] = max_dd
    all_trades_df = pd.concat(all_trades, ignore_index=True) if all_trades else pd.DataFrame()
    return all_trades_df, equity_tracks, max_dd_per_symbol


In [32]:
# Run the swing backtest for the defined STOCKS universe.
# WARNING: This will download data and run the backtest for each ticker.
# Uncomment and run when ready.

trades, equity_tracks, max_dds = backtest_universe_swing(STOCKS)
print('Max drawdowns per symbol (%)', max_dds)
display(trades.head())
print('Swing backtest cell ready. Uncomment the run lines to execute the backtest.')

Running: SBICARD.NS
Running: BDL.NS


/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_94721/3118873791.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['EMA20_gt_EMA50_prev'] = df['EMA20_gt_EMA50'].shift(1).fillna(False)
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_94721/3118873791.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['EMA20_gt_EMA50_prev'] = df['EMA20_gt_EMA50'].shift(1).fillna(False)


Running: INDHOTEL.NS


/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_94721/3118873791.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['EMA20_gt_EMA50_prev'] = df['EMA20_gt_EMA50'].shift(1).fillna(False)


Running: BSE.NS


...

/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_94721/3118873791.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['EMA20_gt_EMA50_prev'] = df['EMA20_gt_EMA50'].shift(1).fillna(False)


Running: BAJAJFINSV.NS
Max drawdowns per symbol (%) {'SBICARD.NS': np.float64(2.731084636042651), 'BDL.NS': np.float64(28.86678800277908), 'INDHOTEL.NS': np.float64(17.336883014302025), 'BSE.NS': np.float64(15.077231392626784), 'NYKAA.NS': np.float64(6.131949735645395), 'BAJFINANCE.NS': np.float64(33.53315592997779), 'PAYTM.NS': np.float64(2.0082951171875028), 'SOLARINDS.NS': np.float64(16.780047680567296), 'CHOLAFIN.NS': np.float64(24.28399215781462), 'UNITDSPR.NS': 0.0, 'DIVISLAB.NS': np.float64(11.048703695447024), 'MUTHOOTFIN.NS': np.float64(5.000003987365392), 'BHARTIARTL.NS': np.float64(9.414087440809501), 'ICICIBANK.NS': np.float64(4.484811488501379), 'MAZDOCK.NS': np.float64(4.564816526882012), 'SHREECEM.NS': np.float64(9.894629477747557), 'DIXON.NS': np.float64(5.61752768482091), 'PERSISTENT.NS': np.float64(13.851123251158326), 'SRF.NS': np.float64(17.003419760143665), 'TVSMOTOR.NS': np.float64(13.32208581936308), 'SBILIFE.NS': np.float64(3.0500006416447993), 'MAXHEALTH.NS': 0

/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_94721/3118873791.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['EMA20_gt_EMA50_prev'] = df['EMA20_gt_EMA50'].shift(1).fillna(False)


,ticker,entry_date,entry_price,side,size,initial_stop,trailing_stop,exit_date,exit_price,pnl,reason
0,SBICARD.NS,2023-07-26,878.524828,long,277.0,860.511061,860.511061,2023-07-27,868.809721,-2711.084636,trailing_stop_hit
1,BDL.NS,2018-06-04,176.317174,long,1020.0,171.419265,171.419265,2018-06-05,172.644339,-3766.291853,trailing_stop_hit
2,BDL.NS,2018-06-07,168.295453,short,1020.0,173.010989,173.010989,2018-06-08,175.048370,-6907.974960,trailing_stop_hit
3,BDL.NS,2018-06-25,166.415998,short,1102.0,170.463403,168.089372,2018-06-27,165.510629,977.717103,trailing_stop_hit
4,BDL.NS,2019-12-16,133.591681,short,750.0,139.603730,134.584609,2019-12-27,140.686367,-5341.014542,trailing_stop_hit


Swing backtest cell ready. Uncomment the run lines to execute the backtest.


In [33]:
# Performance summary helpers
def summary_from_trades(trades_df, equity_df, initial_capital=INITIAL_CAPITAL):
    total_trades = len(trades_df)
    wins = trades_df[trades_df['pnl']>0]
    losses = trades_df[trades_df['pnl']<=0]
    win_rate = len(wins)/total_trades if total_trades>0 else np.nan
    total_pnl = trades_df['pnl'].sum() if total_trades>0 else 0.0
    avg_win = wins['pnl'].mean() if len(wins)>0 else 0.0
    avg_loss = losses['pnl'].mean() if len(losses)>0 else 0.0
    final_equity = equity_df['equity'].iloc[-1] if not equity_df.empty else initial_capital
    total_return = (final_equity - initial_capital)/initial_capital * 100
    rolling_max = equity_df['equity'].cummax()
    drawdown = (rolling_max - equity_df['equity']) / rolling_max
    max_dd = drawdown.max() * 100 if not drawdown.empty else 0.0
    return {'total_trades':total_trades,'win_rate':win_rate,'total_pnl':total_pnl,'avg_win':avg_win,'avg_loss':avg_loss,'final_equity':final_equity,'total_return_pct':total_return,'max_dd_pct':max_dd}


In [34]:
# Plot equity curve for a symbol (after running backtest)
def plot_equity(equity_df, title='Equity Curve'):
    plt.figure(figsize=(12,5))
    plt.plot(equity_df.index, equity_df['equity'])
    plt.title(title)
    plt.xlabel('Date')
    plt.ylabel('Equity')
    plt.grid(True)
    plt.show()


## Notes & Next steps

- This notebook is configured for swing trading on daily bars and uses next-day open execution for entries/exits to avoid intraday fills.
- Risk per trade is set to 5% of capital (RISK_PER_TRADE). Change parameters to tune.
- Trailing stop is ATR-based and only moves favorably; stops trigger exit at next-day open when breached during a day's low/high.
- For portfolio-level shared capital allocation or multiple simultaneous positions, the engine must be extended to manage a global capital pool.
